In [1]:
import sys # aby importy z dołu działały
sys.path.append(r"D:\MyProjects_4Fun\projects\World of Warcraft\python-etl")

In [2]:
from sqlalchemy import text
from moduly.db_core import utworz_engine_do_db, _czy_duplikat
from moduly.services_persist_wynik import przefiltruj_dane_misji, zapisz_misje_dialogi_ai_do_db
from moduly.ai_gemini import zaladuj_api_i_klienta, instrukcja_tlumacz, instrukcja_redaktor
from sqlalchemy.exc import IntegrityError
from scraper_wiki_main import parsuj_misje_z_url
from google import genai
import json
import zlib
import base64
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint

In [ ]:
silnik = utworz_engine_do_db()
klient = zaladuj_api_i_klienta("API_TLUMACZENIE")

sciezka = r"D:\MyProjects_4Fun\projects\World of Warcraft\excel-mappingi\npc.xlsx"
sciezka_zapis = r"D:\MyProjects_4Fun\projects\World of Warcraft\excel-mappingi\surowe\propozycja_tlumaczen_npc"
zakladka = "surowe"

df = pd.read_excel(sciezka, sheet_name=zakladka, usecols=["NPC_ID_MOJE_PK", "NAZWA", "NAZWA_PL_FINAL"], index_col="NPC_ID_MOJE_PK")
df = df.loc[df["NAZWA_PL_FINAL"].isna()]

co_ile = 100
probowka = df.head(200)

for i in range(0, len(probowka), co_ile):
    start = i
    koniec = i + co_ile
    
    batch_seria = probowka["NAZWA"].iloc[start:koniec]
    dfj = batch_seria.to_json(force_ascii=False)

    instrukcja = """
        Jesteś ekspertem od uniwersum World of Warcraft. 
        Analizujesz listę NPC (ID: NAZWA_ANGIELSKA). 
        Twoim jedynym zadaniem jest zwrócenie JSONa zawierającego WYŁĄCZNIE te wpisy, których NIE NALEŻY tłumaczyć na język polski.

        Kryteria pozostawienia w oryginale:
        1. Pojedyncze imiona własne (np. Agatha, Om'sirik, Orwenya).
        2. Pełne imiona i nazwiska (np. Jaina Proudmoore, Corithras Moonrage).
        
        Kogo NIE zwracaj:
        1. Nazw z tytułami, przydomkami lub nazwami pospolitymi (np. Sergeant Willem, Jack the Hammer, Stormwind Guard). 
           Te nazwy będę tłumaczył później, więc ich tutaj NIE umieszczaj.

        Zwróć wyłącznie poprawny JSON w tej samej strukturze (ID: NAZWA). 
        Jeśli w danej paczce wszyscy NPC wymagają tłumaczenia, zwróć pusty słownik {}.
    """

    odpowiedz = klient.models.generate_content(
        model="gemini-3-pro-preview",
        contents=dfj,
        config={
            "system_instruction": instrukcja, 
            "response_mime_type": "application/json"
        }
    )
    
    zaladowane = json.loads(odpowiedz.text)

    df_wynikowy = pd.DataFrame.from_dict(zaladowane, orient="index", columns=["NAZWA"])
    df_wynikowy.to_csv("tylko_odrzuty.csv", sep=";", encoding="utf-8-sig", index_label="NPC_ID_MOJE_PK")

In [16]:
zaladowane

{'854': 'Melithar Staghelm',
 '859': 'Moraya',
 '860': 'Kijara',
 '862': 'Lanthan Perilon',
 '864': 'Morakki',
 '868': "Zar'jira",
 '869': 'Vanira',
 '904': 'Tolaan',
 '908': 'Proenitus',
 '910': 'Tortunga',
 '918': 'Tegashi',
 '920': "Tora'jin",
 '939': 'Agatha',
 '1018': 'Zalduun',
 '1020': 'Falkhaan Isenstrider',
 '1061': 'Corithras Moonrage',
 '1158': 'Megelon',
 '1168': 'Frang',
 '1172': 'Rwag',
 '1820': 'Jartsam',
 '1841': 'Orwenya',
 '1852': 'Alton Parsons',
 '1853': 'Galtra Bloodcleave',
 '2043': 'Galtra Bloodaxe',
 '2067': "Ve'nari",
 '2075': "Om'sirik",
 '2082': 'Alleria Winderunner',
 '2087': "Om'talad",
 '2090': 'Lindormi',
 '2094': 'Soridormi',
 '2095': 'Augermu',
 '2102': 'Val Thalvassa',
 '2107': "Ar'syn",
 '2111': 'Rakva',
 '2114': "Zo'ya",
 '2122': 'Slagg',
 '2123': 'Drum Fel',
 '2136': 'Karga Bloodfury',
 '2145': "Ct'nak",
 '2147': 'Aliya Hillhelm',
 '2148': 'Keyrra Flamestonge',
 '2151': 'Adelgonn',
 '2152': 'Shahata',
 '2154': "Om'fennad",
 '2158': 'Jazz Mokkitz',
 